In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, isnan, when, count, isnull

In [3]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
spark = SparkSession.builder \
    .appName("Flight Data Analysis") \
    .getOrCreate()

spark.conf.set("spark.sql.debug.maxToStringFields", 1000)
spark.sparkContext.setLogLevel("ERROR")

25/04/25 14:54:18 WARN Utils: Your hostname, Mihirs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
25/04/25 14:54:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 14:54:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import glob
import os
os.getcwd()
home_dir = os.path.expanduser('~')
folder_path = os.path.join(home_dir, 'Desktop/GroupProject/data/archive/raw')
# folder_path = '~/Desktop/GroupProject/data/archive/raw'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
df = spark.read.csv(csv_files,
                       sep = ',',
                       inferSchema = True,
                       header = True)


In [6]:
num_entries = 29193782

In [7]:
null_counts = df.select([count(col(c)).alias(c) for c in df.columns]).collect()[0].asDict()
print(null_counts)

[Stage 4:======================================================>(115 + 1) / 116]

{'Year': 29193782, 'Quarter': 29193782, 'Month': 29193782, 'DayofMonth': 29193782, 'DayOfWeek': 29193782, 'FlightDate': 29193782, 'Marketing_Airline_Network': 29193782, 'Operated_or_Branded_Code_Share_Partners': 29193782, 'DOT_ID_Marketing_Airline': 29193782, 'IATA_Code_Marketing_Airline': 29193782, 'Flight_Number_Marketing_Airline': 29193782, 'Originally_Scheduled_Code_Share_Airline': 3873, 'DOT_ID_Originally_Scheduled_Code_Share_Airline': 3873, 'IATA_Code_Originally_Scheduled_Code_Share_Airline': 3873, 'Flight_Num_Originally_Scheduled_Code_Share_Airline': 3873, 'Operating_Airline ': 29193782, 'DOT_ID_Operating_Airline': 29193782, 'IATA_Code_Operating_Airline': 29193782, 'Tail_Number': 28926171, 'Flight_Number_Operating_Airline': 29193782, 'OriginAirportID': 29193782, 'OriginAirportSeqID': 29193782, 'OriginCityMarketID': 29193782, 'Origin': 29193782, 'OriginCityName': 29193782, 'OriginState': 29193782, 'OriginStateFips': 29193782, 'OriginStateName': 29193782, 'OriginWac': 29193782, 'D

In [ ]:
columns_with_few_nulls = [col_name for col_name, count_val in null_counts.items() if count_val > 0.9*num_entries]
#columns_with_no_nulls = [col_name for col_name, count_val in null_counts.items() if count_val != 0]
print(columns_with_few_nulls)
#print(columns_with_all_nulls)
#df.select(*columns_with_nulls).show()

In [ ]:
newdf = df.select(columns_with_few_nulls)
newdf.select(newdf.columns[0:10]).show()

In [ ]:
newdf.select(newdf.columns[10:19]).show()

In [ ]:
newdf.select(newdf.columns[19:33]).show()

In [ ]:
newdf.select(newdf.columns[33:50]).show()

In [ ]:
newdf.select(newdf.columns[50:62]).show()

In [ ]:
repeat_cols = ["Marketing_Airline_Network", "Operated_or_Branded_Code_Share_Partners", "IATA_Code_Marketing_Airline", "Operating_Airline ", "IATA_Code_Operating_Airline"]
print(newdf.select([count(col(c)).alias(c) for c in repeat_cols]).collect()[0].asDict())

In [ ]:
print(newdf.select(repeat_cols).distinct().count())

In [ ]:
cols_to_keep = ["Year", "Month", "DayofMonth", "Origin", "OriginCityName", "DestCityName", "DepDelay", "ArrDelay", "Cancelled", "CRSElapsedTime", "ActualElapsedTime"]
my_df = newdf.select(cols_to_keep)
my_df.show()

## Which Origin Cities had the most delayed flights?

In [ ]:
count_delay = my_df.select(["Origin", "DepDelay"]).groupBy("Origin")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount").desc())
pandas_delay = count_delay.toPandas()

In [ ]:
pdf = pandas_delay.copy()
pdf["OnTimeCount"] = pdf["TotalCount"] - pdf["DelayCount"] - pdf["EarlyCount"]

In [ ]:
top_20 = pdf.head(20)
top_20

In [ ]:
count_delay = my_df.select(["Origin", "DepDelay"]).groupBy("Origin")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount").desc())
pandas_delay = count_delay.toPandas()# Assuming pdf has these columns: OriginCity, DelayedFlights, EarlyFlights, OnTimeFlights

# Bar positions
cities = top_20["Origin"]
x = np.arange(len(cities))

# Heights
early = top_20["EarlyCount"]
on_time = top_20["OnTimeCount"]
delayed = top_20["DelayCount"]

# Plot
plt.figure(figsize=(12, 6))
plt.bar(x, early, label="Early", color="green")
plt.bar(x, on_time, bottom=early, label="On Time", color="gray")
plt.bar(x, delayed, bottom=early + on_time, label="Delayed", color="red")

# Labels and formatting
plt.xticks(x, cities, rotation=45)
plt.ylabel("Number of Flights")
plt.title("Flight Status by Origin City (Top 20)")
plt.legend(title="Flight Status")
plt.tight_layout()
plt.show()

In [ ]:
year_delay = my_df.select(["Year", "DepDelay"]).groupBy("Origin")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount").desc())
pandas_year_delay = count_delay.toPandas()